In [28]:
import pandas as pd
balanced_labels = pd.read_csv('./chosen_labels_not_all_rules.csv', sep=';',index_col=False)
predictions_labels = pd.read_csv("./predicted_labels_not_all_rules.csv", sep=";", index_col=False)

In [29]:
df_filtered = balanced_labels.drop_duplicates(subset=['Image_name'],keep="first")
predictions_labels=predictions_labels.drop_duplicates(subset=['Image_name'],keep="first")

In [30]:
predictions_labels.describe()

,Rule1,Rule2,Rule5,Rule6,Rule7
count,809.000000,809.000000,809.000000,809.000000,809.000000
mean,0.174289,0.187886,0.269468,0.393078,0.152040
std,0.379593,0.390863,0.443958,0.488736,0.359281
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
df_filtered.describe()

,Rule1,Rule2,Rule5,Rule6,Rule7
count,809.000000,809.000000,809.000000,809.000000,809.000000
mean,0.175525,0.164400,0.186650,0.325093,0.055624
std,0.380651,0.370868,0.389871,0.468699,0.229336
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
tn = 0
fp = 0
fn = 0
tp = 0
all_zero = 0

for index, row in df_filtered.iterrows():
    image = row['Image_name']
    
    # Extrahiere die tatsächlichen Werte ab der zweiten Spalte
    actual_value_list = row[1:]
    if all(x == 0 for x in actual_value_list):
        actual_value = 0
    else:
        actual_value = 1

    # Finde die Vorhersagezeile für das Bild
    predicted_row = predictions_labels[predictions_labels["Image_name"] == image]
    
    if not predicted_row.empty:
        # Extrahiere die vorhergesagten Werte ab der zweiten Spalte
        predicted_value_list = predicted_row.iloc[0, 1:]
        
        if all(x == 0 for x in predicted_value_list):
            predicted_value = 0
            all_zero += 1
        else:
            predicted_value = 1

        # Aktualisiere die Zähler basierend auf den tatsächlichen und vorhergesagten Werten
        if actual_value == 0 and predicted_value == 0:
            tn += 1
        elif actual_value == 0 and predicted_value == 1:
            fp += 1
        elif actual_value == 1 and predicted_value == 0:
            fn += 1
        elif actual_value == 1 and predicted_value == 1:
            tp += 1
    else:
        print(f"Keine Vorhersage für Bild: {image}")

print(f"TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}")
print(all_zero/len(df_filtered))


TN: 237, FP: 57, FN: 38, TP: 477
0.3399258343634116


In [27]:
err = (fp+fn)/(tp+tn+fn+fp)
acc = (tp+tn)/(tp+tn+fn+fp)
f1 = (2*tp)/(2*tp+fp+fn)
precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("We want to minimize false positives, so we want to maximize precision.")
print(f"Anzahl Bilder: {len(df_filtered)}\nError rate: {round(err*100, 2)}\nAccuracy: {round(acc*100, 2)}\nF1: {round(f1,2)}\nPrecision: {round(precision,2)}\nRecall: {round(recall,2)}")

We want to minimize false positives, so we want to maximize precision.
Anzahl Bilder: 809
Error rate: 11.74
Accuracy: 88.26
F1: 0.91
Precision: 0.89
Recall: 0.93
